In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from src.utilities import *
from src.features.activity_feature import extract_activity_features

In [6]:
raw_activity = pd.read_csv(os.path.join(raw_path, 'activity.csv'))
raw_activity=raw_activity.rename(columns = {'count':'counts'})
raw_activity.head()

,hcp,specialty,region,brand,month,channel,counts
0,39972,Internal medicine / pneumology,region_61,brand_1,2020-05,video,1
1,64026,General practicioner,region_168,brand_2,2020-05,phone,1
2,27892,Internal medicine,region_26,brand_2,2020-06,phone,1
3,33500,General practicioner,region_26,brand_2,2020-06,phone,1
4,61239,General practicioner,region_26,brand_2,2020-06,phone,1


In [30]:
def add_month(month):
    '''Given a string with month format of YYYY-MM add one month'''

    if (int(month[5:])<12):
        if (int(month[5:]) + 1 >= 10):
            return month[:5] + str(int(month[5:]) + 1)
        return month[:5] + '0' + str(int(month[5:]) + 1)
    return str(int(month[:4]) + 1) + '-01'

In [9]:
activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                          'specialty']).counts.sum().unstack().reset_index().fillna(0)
activity_features_specialty.isna().sum()

specialty
month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
dtype: int64

In [10]:
#activities to different specialty per month
activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                          'specialty']).counts.sum().unstack().reset_index().fillna(0)
activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['General practicioner'].sum())
activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine'].sum())
activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Pediatrician'].sum())

In [12]:
activity_features_specialty

specialty
month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
dtype: int64

In [15]:
#activities via different channel per month
activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                          'channel']).counts.sum().unstack().reset_index().fillna(0)
activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['f2f'].sum())
activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['other'].sum())
activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['phone'].sum())
activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['video'].sum())
activity_features_channel.isna().sum()

channel
month            0
region           0
brand            0
f2f              0
other            0
phone            0
video            0
act_f2f_1mo      0
act_other_1mo    0
act_phone_1mo    0
act_video_1mo    0
dtype: int64

In [17]:
#activity to different specialty per month per region
GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

activity_features_specialty.isna().sum()

month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
act_GP_region_1mo_x                           0
act_IM_region_1mo_x                           0
act_IMP_region_1mo_x                          0
act_IMGP_region_1mo_x                         0
act_PE_region_1mo_x                           0
act_GP_region_1mo_y                           0
act_IM_region_1mo_y                     

In [18]:
#activity via different channel per month per region
f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])

activity_features_channel.isna().sum()

month                   0
region                  0
brand                   0
f2f                     0
other                   0
phone                   0
video                   0
act_f2f_1mo             0
act_other_1mo           0
act_phone_1mo           0
act_video_1mo           0
act_f2f_region_1mo      0
act_phone_region_1mo    0
act_video_region_1mo    0
act_other_region_1mo    0
dtype: int64

In [30]:
# groupby = master.value_counts(['month', 'brand']).reset_index(name = 'monthly brand activity')
# master['monthly brand activity'] = groupby['monthly brand activity']

In [19]:
#activity to different specialty per month per brand
GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_specialty.isna().sum()

month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
act_GP_region_1mo_x                           0
act_IM_region_1mo_x                           0
act_IMP_region_1mo_x                          0
act_IMGP_region_1mo_x                         0
act_PE_region_1mo_x                           0
act_GP_region_1mo_y                           0
act_IM_region_1mo_y                     

In [20]:
#activity via different channel per month per brand
f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_channel.isna().sum()

month                   0
region                  0
brand                   0
f2f                     0
other                   0
phone                   0
video                   0
act_f2f_1mo             0
act_other_1mo           0
act_phone_1mo           0
act_video_1mo           0
act_f2f_region_1mo      0
act_phone_region_1mo    0
act_video_region_1mo    0
act_other_region_1mo    0
act_f2f_brand_1mo       0
act_phone_brand_1mo     0
act_video_brand_1mo     0
act_other_brand_1mo     0
dtype: int64

In [22]:
activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])



In [23]:
activity_features.isna().sum()

month                                         0
region                                        0
brand                                         0
f2f                                           0
other                                         0
phone                                         0
video                                         0
act_f2f_1mo                                   0
act_other_1mo                                 0
act_phone_1mo                                 0
act_video_1mo                                 0
act_f2f_region_1mo                            0
act_phone_region_1mo                          0
act_video_region_1mo                          0
act_other_region_1mo                          0
act_f2f_brand_1mo                             0
act_phone_brand_1mo                           0
act_video_brand_1mo                           0
act_other_brand_1mo                           0
General practicioner                          0
Internal medicine                       

In [24]:
activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})

In [25]:
activity_features.isna().sum()

month                         0
region                        0
brand                         0
act_f2f_region_brand_1mo      0
act_other_region_brand_1mo    0
act_phone_region_brand_1mo    0
act_video_region_brand_1mo    0
act_f2f_1mo                   0
act_other_1mo                 0
act_phone_1mo                 0
act_video_1mo                 0
act_f2f_region_1mo            0
act_phone_region_1mo          0
act_video_region_1mo          0
act_other_region_1mo          0
act_f2f_brand_1mo             0
act_phone_brand_1mo           0
act_video_brand_1mo           0
act_other_brand_1mo           0
act_GP_region_brand_1mo       0
act_IM_region_brand_1mo       0
act_IMP_region_brand_1mo      0
act_IMPG_region_brand_1mo     0
act_PE_region_brand_1mo       0
act_total_1mo                 0
act_GP_1mo                    0
act_IM_1mo                    0
act_IMP_1mo                   0
act_IMGP_1mo                  0
act_PE_1mo                    0
act_GP_region_1mo_x           0
act_IM_r

In [52]:
def extract_activity_features(raw_activity: pd.DataFrame) -> pd.DataFrame:

    raw_activity.sort_values('month', inplace = True)
    
    #activities to different specialty per month
    activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                              'specialty']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
    activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['General practicioner'].sum())
    activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine'].sum())
    activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
    activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
    activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Pediatrician'].sum())
    
    #activities via different channel per month
    activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                              'channel']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['f2f'].sum())
    activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['other'].sum())
    activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['phone'].sum())
    activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['video'].sum())
    activity_features_channel

    #activity to different specialty per month per region
    GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
    GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
    IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
    IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
    IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
    IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
    PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
    PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

    #activity to different specialty per month per brand
    GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
    GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
    IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
    IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
    IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
    IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
    PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
    PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

    #activity via different channel per month per region
    f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
    f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
    phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
    phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
    video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
    video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
    other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
    other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])



    #activity via different channel per month per brand
    f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
    f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
    phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
    phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
    video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
    video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
    other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
    other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

    
    activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])


    activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})
    
    activity_features.month = activity_features.month.apply(lambda x: add_month(x))

    # Add rolling aggregations of the previously created features
    cols = [c for c in activity_features.columns if c not in ['month', 'region', 'brand']]
    
    for c in cols:
        activity_features[c[:-3] + '2mo'] = activity_features.groupby(['region', 'brand'])[c].shift()
        activity_features[c[:-3] + '3mo'] = activity_features.groupby(['region', 'brand'])[c].shift(2)
        activity_features[c[:-3] + '4mo'] = activity_features.groupby(['region', 'brand'])[c].shift(3)
        
        activity_features[ c[:-3] + 'trend_2mo'] = activity_features[c[:-3] + '2mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[c[:-3] + 'trend_3mo'] = activity_features[c[:-3] + '3mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[ c[:-3] + 'trend_4mo'] = activity_features[c[:-3] + '4mo'] / (activity_features[c[:-3] + '1mo']+1)

    months = pd.DataFrame({'month': raw_activity.month.unique()})
    regions = pd.DataFrame({'region': raw_activity.region.unique()})
    brands = pd.DataFrame({'brand': ['brand_1', 'brand_2']})

    months['dummy_col'] = 0
    regions['dummy_col'] = 0
    brands['dummy_col'] = 0

    activity_features_full = months.merge(regions, how = 'outer', on = 'dummy_col')
    activity_features_full = activity_features_full.merge(brands, how = 'outer', on = 'dummy_col')
    activity_features_full.drop(columns = 'dummy_col', inplace = True)
    activity_features_full.sort_values('month', inplace = True)
    activity_features = activity_features.merge(activity_features_full,how='outer',on=['month','region','brand'])
    activity_features = activity_features.fillna(0)
    return activity_features


In [58]:
#checking 
raw_activity[(raw_activity.month=='2020-10')&(raw_activity.region=='region_70')&
             (raw_activity.channel=='f2f')&(raw_activity.brand=='brand_1')].counts.sum()

22

In [57]:
#checking
temp = extract_activity_features(raw_activity)
temp[(temp.brand=='brand_1')&(temp.region=='region_70')][['month', 'brand', 'region', 
                                                           'act_f2f_region_brand_1mo','act_f2f_region_brand_2mo', 'act_f2f_region_brand_3mo']]

,month,brand,region,act_f2f_region_brand_1mo,act_f2f_region_brand_2mo,act_f2f_region_brand_3mo
365,2020-09,brand_1,region_70,25.0,0.0,0.0
752,2020-10,brand_1,region_70,11.0,25.0,0.0
1145,2020-11,brand_1,region_70,22.0,11.0,25.0
1518,2020-12,brand_1,region_70,0.0,22.0,11.0
1804,2021-01,brand_1,region_70,0.0,0.0,22.0
2294,2021-03,brand_1,region_70,0.0,0.0,0.0
2539,2021-04,brand_1,region_70,5.0,0.0,0.0
2772,2021-05,brand_1,region_70,0.0,5.0,0.0
3000,2021-06,brand_1,region_70,2.0,0.0,5.0
3301,2021-07,brand_1,region_70,9.0,2.0,0.0


In [50]:
result = extract_activity_features(raw_activity)
result.groupby(['month']).size()

month
2020-05    402
2020-06    402
2020-07    402
2020-08    402
2020-09    402
2020-10    402
2020-11    402
2020-12    402
2021-01    402
2021-02    402
2021-03    402
2021-04    402
2021-05    402
2021-06    402
2021-07    402
2021-08    402
2021-09    402
2021-10    307
dtype: int64

In [53]:
extract_activity_features(raw_activity).groupby('month').apply(lambda x: x.isna().mean())

,month,region,brand,act_f2f_region_brand_1mo,act_other_region_brand_1mo,act_phone_region_brand_1mo,act_video_region_brand_1mo,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo,act_f2f_region_1mo,act_phone_region_1mo,act_video_region_1mo,act_other_region_1mo,act_f2f_brand_1mo,act_phone_brand_1mo,act_video_brand_1mo,act_other_brand_1mo,act_GP_region_brand_1mo,act_IM_region_brand_1mo,act_IMP_region_brand_1mo,act_IMPG_region_brand_1mo,act_PE_region_brand_1mo,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo,act_GP_brand_1mo,act_IM_brand_1mo,act_IMP_brand_1mo,act_IMGP_brand_1mo,act_PE_brand_1mo,act_f2f_region_brand_2mo,act_f2f_region_brand_3mo,act_f2f_region_brand_4mo,act_f2f_region_brand_trend_2mo,act_f2f_region_brand_trend_3mo,act_f2f_region_brand_trend_4mo,act_other_region_brand_2mo,act_other_region_brand_3mo,act_other_region_brand_4mo,act_other_region_brand_trend_2mo,act_other_region_brand_trend_3mo,act_other_region_brand_trend_4mo,act_phone_region_brand_2mo,act_phone_region_brand_3mo,act_phone_region_brand_4mo,act_phone_region_brand_trend_2mo,act_phone_region_brand_trend_3mo,act_phone_region_brand_trend_4mo,act_video_region_brand_2mo,act_video_region_brand_3mo,act_video_region_brand_4mo,act_video_region_brand_trend_2mo,act_video_region_brand_trend_3mo,act_video_region_brand_trend_4mo,act_f2f_2mo,act_f2f_3mo,act_f2f_4mo,act_f2f_trend_2mo,act_f2f_trend_3mo,act_f2f_trend_4mo,act_other_2mo,act_other_3mo,act_other_4mo,act_other_trend_2mo,act_other_trend_3mo,act_other_trend_4mo,act_phone_2mo,act_phone_3mo,act_phone_4mo,act_phone_trend_2mo,act_phone_trend_3mo,act_phone_trend_4mo,act_video_2mo,act_video_3mo,act_video_4mo,act_video_trend_2mo,act_video_trend_3mo,act_video_trend_4mo,act_f2f_region_2mo,act_f2f_region_3mo,act_f2f_region_4mo,act_f2f_region_trend_2mo,act_f2f_region_trend_3mo,act_f2f_region_trend_4mo,act_phone_region_2mo,act_phone_region_3mo,act_phone_region_4mo,act_phone_region_trend_2mo,act_phone_region_trend_3mo,act_phone_region_trend_4mo,act_video_region_2mo,act_video_region_3mo,act_video_region_4mo,act_video_region_trend_2mo,act_video_region_trend_3mo,act_video_region_trend_4mo,act_other_region_2mo,act_other_region_3mo,act_other_region_4mo,act_other_region_trend_2mo,act_other_region_trend_3mo,act_other_region_trend_4mo,act_f2f_brand_2mo,act_f2f_brand_3mo,act_f2f_brand_4mo,act_f2f_brand_trend_2mo,act_f2f_brand_trend_3mo,act_f2f_brand_trend_4mo,act_phone_brand_2mo,act_phone_brand_3mo,act_phone_brand_4mo,act_phone_brand_trend_2mo,act_phone_brand_trend_3mo,act_phone_brand_trend_4mo,act_video_brand_2mo,act_video_brand_3mo,act_video_brand_4mo,act_video_brand_trend_2mo,act_video_brand_trend_3mo,act_video_brand_trend_4mo,act_other_brand_2mo,act_other_brand_3mo,act_other_brand_4mo,act_other_brand_trend_2mo,act_other_brand_trend_3mo,act_other_brand_trend_4mo,act_GP_region_brand_2mo,act_GP_region_brand_3mo,act_GP_region_brand_4mo,act_GP_region_brand_trend_2mo,act_GP_region_brand_trend_3mo,act_GP_region_brand_trend_4mo,act_IM_region_brand_2mo,act_IM_region_brand_3mo,act_IM_region_brand_4mo,act_IM_region_brand_trend_2mo,act_IM_region_brand_trend_3mo,act_IM_region_brand_trend_4mo,act_IMP_region_brand_2mo,act_IMP_region_brand_3mo,act_IMP_region_brand_4mo,act_IMP_region_brand_trend_2mo,act_IMP_region_brand_trend_3mo,act_IMP_region_brand_trend_4mo,act_IMPG_region_brand_2mo,act_IMPG_region_brand_3mo,act_IMPG_region_brand_4mo,act_IMPG_region_brand_trend_2mo,act_IMPG_region_brand_trend_3mo,act_IMPG_region_brand_trend_4mo,act_PE_region_brand_2mo,act_PE_region_brand_3mo,act_PE_region_brand_4mo,act_PE_region_brand_trend_2mo,act_PE_region_brand_trend_3mo,act_PE_region_brand_trend_4mo,act_total_2mo,act_total_3mo,act_total_4mo,act_total_trend_2mo,act_total_trend_3mo,act_total_trend_4mo,act_GP_2mo,act_GP_3mo,act_GP_4mo,act_GP_trend_2mo,act_GP_trend_3mo,act_GP_trend_4mo,act_IM_2mo,act_IM_3mo,act_IM_4mo,act_IM_trend_2mo,act_IM_trend

In [59]:
raw_activity['brand_region'] = raw_activity['brand'] + raw_activity['region']
raw_activity.groupby(['month']).brand_region.nunique()

month
2020-05      2
2020-06      7
2020-07    201
2020-08    375
2020-09    393
2020-10    384
2020-11    320
2020-12    252
2021-01    246
2021-02    242
2021-03    257
2021-04    199
2021-05    275
2021-06    335
2021-07    340
2021-08    302
2021-09    307
Name: brand_region, dtype: int64